<h1><center>CSCI 3155 Principles of Programming Languages</center></h1>
<h2><center>Spring 2025</center></h2>

## Functions & Recursion in Lettuce


$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & True \\
 & | & False \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\[5pt]
 & | & LetRec( \mathbf{Identifier}, \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{letrec identifier = function(identifier) expr in expr} \\
\end{array}$$

~~~
LetRec("factorial","n",if(n=0) then 1 else n*factorial(n-1), factorial(3))
~~~

~~~
letrec factorial = function(n) 
                        if(n=0) then 1 
                        else n*factorial(n-1)
                    in
  factorial(3)
~~~                    

In [1]:
sealed trait Program
sealed trait Expr

case class TopLevel(e: Expr) extends Program

case class Const(v: Double) extends Expr // Expr -> Const(v)
case object True extends Expr // Expr -> True
case object False extends Expr // Expr -> False
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)
case class Minus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)
case class Mult(e1: Expr, e2: Expr) extends Expr // Expr -> Mult (Expr, Expr)

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2:Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

//LetRec bindings
case class LetRec(funName: String, param:String, funBody: Expr, innerExpr: Expr) extends Expr

//Function definition
case class FunDef(param: String, bodyExpr: Expr) extends Expr

// Function call
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr

defined trait Program
defined trait Expr
defined class TopLevel
defined class Const
defined object True
defined object False
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Geq
defined class Eq
defined class IfThenElse
defined class Let
defined class LetRec
defined class FunDef
defined class FunCall

In [2]:
/* 1. Define the values */
sealed trait Value 
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
/* -- Let us add Closure to the set of values --*/
case class Closure(x: String, e: Expr, env: Environment) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

sealed trait Environment
case object EmptyEnv extends Environment
case class Extend(ident:String, v:Value, env:Environment) extends Environment
case class ExtendRec(funName:String, param:String, body:Expr, env:Environment) extends Environment

def lookup(id:String, env:Environment) : Value = {
    env match {
        case EmptyEnv => throw new IllegalArgumentException("Don't care")
        case Extend(id1,v1, envTail) => if (id1 == id) v1 else lookup(id, envTail)
        case ExtendRec(funName, param, funBody, envTail) => {
            if (funName == id) {
                Closure(param, funBody, env)
            } 
            else lookup(id, envTail)
        }
    }
}

def contains(id:String, env:Environment) : Boolean = {
    env match {
        case EmptyEnv => false
        case Extend(id1,v1, envTail) => if (id1 == id) true else contains(id, envTail)
        case ExtendRec(funName, param, funBody, envTail) => {
            if (funName == id) {
                true
            } 
            else contains(id, envTail)
        }
    }
}


defined trait Value
defined class NumValue
defined class BoolValue
defined class Closure
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined trait Environment
defined object EmptyEnv
defined class Extend
defined class ExtendRec
defined function lookup
defined function contains

In [3]:
def evalExpr(e: Expr, env: Environment): Value =  {
    
    /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        NumValue(v3)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val v = valueToNumber(evalExpr(e, env))
        val v1 = fun(v)
        NumValue(v1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        BoolValue(v3)
    }
    
   
    e match {
        case Const(f) => NumValue(f)
        
        case Ident(x) => {
            if (contains(x,env)) 
                lookup(x,env)
            else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
        case True => BoolValue(true)
    
        case False => BoolValue(false)
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
        
        case Minus(e1, e2) => applyArith2 (e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
    
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
        
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
    
        case IfThenElse(e1, e2, e3) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(true) => evalExpr(e2, env)
                case BoolValue(false) => evalExpr(e3, env)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
    
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)  // eval e1
            val env2 = Extend(x,v1,env) // create a new extended env
            evalExpr(e2, env2) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            Closure(x, e, env)
        }
        
        case FunCall(e1, e2) => {
            val v1 = evalExpr(e1,env)
            v1 match {
                case Closure(x,bodyExpr,oldEnv) => {
                    val v2 = evalExpr(e2, env)
                    v2 match {
                        case ErrorValue => ErrorValue
                        case _ => {
                            val env2 = Extend(x,v2, oldEnv)
                            evalExpr(bodyExpr, env2)
                        }
                    }
                }
                case _ => ErrorValue
            }
            
        }
        
        case LetRec(funName, param, funBody, e2) => {
            val env2 = ExtendRec(funName, param, funBody, env)
            evalExpr(e2, env2)
        }
    }
}

def evalProgram(p: Program) = {
    val m: Environment = EmptyEnv
    p match { 
        case TopLevel(e) => evalExpr(e, m)
    }
}

defined function evalExpr
defined function evalProgram

In [4]:
val p1 = TopLevel( 
    Let("square",                                // let square = 
         FunDef("x", Mult(Ident("x"), Ident("x"))),  //    function (x) x * x
         FunCall(Ident("square"), Const(10)) //     in  square(10)
       )
)

val p1_val = evalProgram(p1)
print(p1_val)


//let x = 10 in 
//let y = 15 in 
//let sq1 = function (x) 
//            function (y) 
//               x + y * y
//           in 
//     sq1(x)(y)
val x = Ident("x")
val y = Ident("y")
val fdef_inner = FunDef("y", Plus(x, Mult(y, y)))
val fdef_outer = FunDef("x", fdef_inner)
val call_expr = FunCall(FunCall(Ident("sq1"), x), y)
val sq1_call = Let("sq1", fdef_outer, call_expr)
val lety = Let("y", Const(15), sq1_call)
val letx = Let("x", Const(10), lety)
val p2 = TopLevel(letx)

val p2_val = evalProgram(p2)
print(p2_val)

//let x = 10 in 
//let y = 15 in 
//let z = 20 in
//let sq1 = function (x) 
//            function (y) 
//               x + y * z
//           in
//let z = 7 in
//sq1(x)(y)

val z = Ident("z")
val fdef_inner1 = FunDef("y", Plus(x, Mult(y, z)))
val fdef_outer1 = FunDef("x", fdef_inner1)
val call_expr1 = FunCall(FunCall(Ident("sq1"), x), y)
val inner_let = Let("z", Const(7), call_expr1)
val sq1_call1 = Let("sq1", fdef_outer1, inner_let)
val letz1 = Let("z", Const(20), sq1_call1) 
val lety1 = Let("y", Const(15), letz1)
val letx1 = Let("x", Const(10), lety1)
val p3 = TopLevel(letx1)

val p3_val = evalProgram(p3)
println(p3_val)


NumValue(100.0)NumValue(235.0)NumValue(310.0)


p1: TopLevel = TopLevel(
  e = Let(
    s = "square",
    defExpr = FunDef(
      param = "x",
      bodyExpr = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "x"))
    ),
    bodyExpr = FunCall(
      funCalled = Ident(s = "square"),
      argExpr = Const(v = 10.0)
    )
  )
)
p1_val: Value = NumValue(d = 100.0)
x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
fdef_inner: FunDef = FunDef(
  param = "y",
  bodyExpr = Plus(
    e1 = Ident(s = "x"),
    e2 = Mult(e1 = Ident(s = "y"), e2 = Ident(s = "y"))
  )
)
fdef_outer: FunDef = FunDef(
  param = "x",
  bodyExpr = FunDef(
    param = "y",
    bodyExpr = Plus(
      e1 = Ident(s = "x"),
      e2 = Mult(e1 = Ident(s = "y"), e2 = Ident(s = "y"))
    )
  )
)
call_expr: FunCall = FunCall(
  funCalled = FunCall(funCalled = Ident(s = "sq1"), argExpr = Ident(s = "x")),
  argExpr = Ident(s = "y")
)
sq1_call: Let = Let(
  s = "sq1",
  defExpr = FunDef(
    param = "x",
    bodyExpr = FunDef(
      param = "y",
      bodyExpr = Plus(
        e1 = Id

In [5]:
/*

let factorial = function(n)
                    if n = 0 then 1
                    else n * factorial (n-1)
                in
    factorial(3)
 
*/
val fact = Ident("factorial")
val n = Ident("n")
// if n == 0 then 1 else n * factorial(n-1)
val ite = IfThenElse(Eq(n, Const(0)), Const(1), Mult(n, FunCall(fact, Minus(n, Const(1)))))
val fundef = FunDef("n", ite)
val e = LetRec("factorial", "n", ite, FunCall(fact, Const(3)))
//val e = Let("factorial", fundef, FunCall(fact, Const(3)))
val wrongFactorial = TopLevel(e)

val fact_val = evalProgram(wrongFactorial)
println(fact_val)

NumValue(6.0)


fact: Ident = Ident(s = "factorial")
n: Ident = Ident(s = "n")
ite: IfThenElse = IfThenElse(
  e = Eq(e1 = Ident(s = "n"), e2 = Const(v = 0.0)),
  eIf = Const(v = 1.0),
  eElse = Mult(
    e1 = Ident(s = "n"),
    e2 = FunCall(
      funCalled = Ident(s = "factorial"),
      argExpr = Minus(e1 = Ident(s = "n"), e2 = Const(v = 1.0))
    )
  )
)
fundef: FunDef = FunDef(
  param = "n",
  bodyExpr = IfThenElse(
    e = Eq(e1 = Ident(s = "n"), e2 = Const(v = 0.0)),
    eIf = Const(v = 1.0),
    eElse = Mult(
      e1 = Ident(s = "n"),
      e2 = FunCall(
        funCalled = Ident(s = "factorial"),
        argExpr = Minus(e1 = Ident(s = "n"), e2 = Const(v = 1.0))
      )
    )
  )
)
e: LetRec = LetRec(
  funName = "factorial",
  param = "n",
  funBody = IfThenElse(
    e = Eq(e1 = Ident(s = "n"), e2 = Const(v = 0.0)),
    eIf = Const(v = 1.0),
    eElse = Mult(
      e1 = Ident(s = "n"),
      e2 = FunCall(
        funCalled = Ident(s = "factorial"),
        argExpr = Minus(e1 = Ident(s = "n

~~~
let almost_fact = function(f)
                    function(n)
                        if (n <= 0) then 1
                        else n * f(f)(n-1) 
                  in
let factorial = function(n)
                    almost_fact(almost_fact)(n)
                in
factorial(3)
~~~

In [6]:
def almost_fact(f:( ((..) => Int => Int )=> Int => Int ) => Int => Int)(n:Int): Int = {
    if (n == 0) 1
    else n*f(f)(n-1)
}

(console):1:23 expected ")"
def almost_fact(f:( ((..) => Int => Int )=> Int => Int ) => Int => Int)(n:Int): Int = {
                      ^